In [1]:
import os
import numpy as np
import pandas as pd
import time
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
title_prob_df = pd.read_csv('../predictions/title_prediction.csv', index_col='article_id')
feature_prob_df = pd.read_csv('../predictions/features_prediction.csv', index_col='article_id')
article_body_prob_df = prob_predicts_df.copy()
print(title_prob_df.shape)
print(feature_prob_df.shape)
print(article_body_prob_df.shape)

In [ ]:
matched_title_prob_df = title_prob_df.loc[feature_prob_df.index]
matched_ids = title_prob_df.index.isin(matched_title_prob_df.index)
unmatched_title_prob_df = title_prob_df[~matched_ids]
print(matched_title_prob_df.shape)
print(unmatched_title_prob_df.shape)

# Late fusion
Combine the prediction from three classifiers

In [ ]:
## Generate multiple sets of weights for text, description, and hashtags
w_title_range = np.arange(0.0, 0.6, 0.01)
w_article_body_range = np.arange(0, 0.4, 0.01)
weight_sets = []

for w_article_body in w_article_body_range:
    for w_title in w_title_range:
        weight_sets.append([w_title, w_article_body, 1 - w_title - w_article_body])
weight_sets[:5]

In [ ]:
## Search for optimal weights to be assigned to individual classifier
y_num = np.array([int(cls[:-1]) for cls in y])
accuracies = []
for i in range(len(weight_sets)):
    w_title, w_article_body, w_features = weight_sets[i]
    combined_prob = w_title * matched_title_prob_df.values + w_article_body * article_body_prob_df.values + w_features * feature_prob_df.values
    combined_pred = np.apply_along_axis(np.argmax, 1, combined_prob)

	avg_p = accuracy_score(y_num, combined_pred)
	avg_r = recall_score(y_num, combined_pred, average='macro')
	avg_f1 = f1_score(y_num, combined_pred, average='macro')

	precisions.append(avg_p)
	recalls.append(avg_r)
	f1_scores.append(avg_f1)